In [4]:
import numpy as np
import torch
import re
import jieba

In [5]:
stop_word_path = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/停词库/stop_word_for_chinese.txt"

def del_element(strings,symbles):
    srcrep = {i:'' for i in symbles }
    rep = dict((re.escape(k), v) for k, v in srcrep.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], strings)
    
def filter_stop_word(strings,stop_word=np.loadtxt(stop_word_path,dtype=str)):
    return del_element(strings,stop_word)

def read_txt(path):
    return open(path,'r').readlines()

def split_word(corpus,n):
    return np.array([word for word in jieba.cut(filter_stop_word(filter_stop_word(read_txt(corpus)[n]),'\n ')) if len(word) > 1 ])

In [6]:
corpus = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/豆瓣电影数据集(2019.3)/豆瓣电影简介.txt"

In [7]:
# 倒入文章并分词
init_paper = np.array([split_word(corpus,i) for i in range(100)])
# 所有单词降维到一维
all_words = np.array([j for i in init_paper for j in i])
# 单词去重
word_vector = np.unique(all_words)
# 测量共有词汇量
m = all_words.size
# 整理词频字典
word_dict = {word:(all_words==word).dot(np.ones(m))/m for word in word_vector}
#word_dict_index = dict(zip(word_vector,range(m)))

Building prefix dict from /usr/local/lib/python3.7/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/sl/q8x6_03132dfk7rktf00yh880000gn/T/jieba.cache
Loading model cost 1.0738599300384521 seconds.
Prefix dict has been built succesfully.


# 倒排表

In [17]:
len(word_dict)

2885

In [8]:
def inverted_index(paper,word_vector):
    result = dict()
    n = -1
    for i in paper:
        n += 1
        for j in i:
            if j in word_vector:
                if j in result:
                    result[j] = result[j]+[n]
                else:
                    result.update({j:[n]})
    return {i:list(set(result[i])) for i in result}

In [9]:
Inverted_Index_List = inverted_index(init_paper,word_vector)
len(Inverted_Index_List)

2885

# 搜索倒排表

In [10]:
def search_inverted_index(strings):
    Inverted_Index_List = inverted_index(init_paper,word_vector)
    words_for_search = []
    split_word_for_search = list(jieba.cut_for_search(strings))
    for word in split_word_for_search:
        if word in Inverted_Index_List:
            print(word,Inverted_Index_List[word])
            words_for_search+=Inverted_Index_List[word]
    return np.array(words_for_search),split_word_for_search

# 词频向量

In [11]:
def word_frequency(words):
    return np.array([word_dict[word] if word in words  else 0 for word in word_dict])

# 余弦相似度

In [12]:
def cosine(s1,s2):
    return s1.dot(s2)/(np.linalg.norm(s1) * np.linalg.norm(s2))

# 搜索倒排表

In [13]:
search_paper_index,search_word = search_inverted_index("北京爱情")
search_paper_index,search_word

北京 [78]
爱情 [60, 92, 62, 84]


(array([78, 60, 92, 62, 84]), ['北京', '爱情'])

# 余弦匹配符合倒排表内容

In [14]:
read_original = read_txt(corpus)
cos = {cosine(word_frequency(paper),word_frequency(search_word)):read_original[index_] for paper,index_ in zip(init_paper[search_paper_index],search_paper_index)}
np.array(sorted(cos.items(), key=lambda x: x[0], reverse=True))[:,1]

array([' 故事发生在上世纪七十年代，寄人篱下的少女何小萍（苗苗饰）经过重重试炼终于加入了梦寐以求的文工团，哪知道这里和她曾经待过的那些地方并无不同，她依然得忍受遭人唾弃和欺侮的生活。唯一给过她温暖的，是刘峰（黄轩饰），不仅仅是对何小萍，被称为“活雷锋”的刘峰把自己的爱情和温暖无私的奉献给每一个人。 \n',
       ' 编号为223的警察（金城武）失恋后患上失恋综合症，在与金发女杀手（林青霞）擦肩而过又离奇相遇并有了一晚温情后，原本以为包括“爱情”在内的所有东西都有保质期的他意外地迎来心灵的短暂温暖。可是，他们的爱情还是结束了。 \n',
       ' 许多年前，亚特兰蒂斯女王（妮可·基德曼NicoleKidman饰）和人类相知相恋，共同孕育了爱情的结晶——后来被陆地人称为海王的亚瑟·库瑞（杰森·莫玛JasonMomoa饰）。在成长的过程中，亚瑟接受海底导师维科（威廉·达福WillemDafoe饰）的严苛训练，时刻渴望去看望母亲，然而作为混血的私生子这却是奢望。与此同时，亚瑟的同母异父兄弟奥姆（帕特里克·威尔森PatrickWilson饰）成为亚特兰蒂斯的国王，他不满陆地人类对大海的荼毒与污染，遂谋划联合其他海底王国发动对陆地的全面战争。为了阻止他的野心，维科和奥姆的未婚妻湄拉（艾梅柏·希尔德AmberHeard饰）将亚瑟带到海底世界。 \n',
       ' 结婚五周年纪念日的早上，尼克·邓恩（本·阿弗莱克BenAffleck饰）来到妹妹玛戈（凯莉·库恩CarrieCoon饰）的酒吧，咒骂抱怨那个曾经彼此恩爱缠绵的妻子艾米（罗莎蒙德·派克RosamundPike饰）以及全然看不见希望的婚姻。当他返回家中时，却发现客厅留下了暴行的痕迹，而妻子竟不见了踪影。女探员朗达·邦妮（金·迪肯斯KimDickens饰）接到报案后赶来调查，而现场留下的种种蛛丝马迹似乎昭示着这并非是一件寻常的失踪案，其背后或许隐藏着裂变于夫妻之情的谋杀罪行。艾米的失踪通过媒体大肆渲染和妄加揣测很快闻名全国，品行不端的尼克被推上风口浪尖，至今不见踪影的爱人对他进行无情审判，你侬我侬的甜言蜜语早已化作以血洗血的复仇与折磨…… \n',
       ' 老炮儿在北京话中，专指提笼遛鸟，无所事事的老混混儿。 \n'], dtype='<U323')

In [15]:
def search(key):
    search_paper_index,search_word = search_inverted_index(key)
    print(search_paper_index)
    read_original = read_txt(corpus)
    cos = {cosine(word_frequency(paper),word_frequency(search_word))    :read_original[index_] for paper,index_ in zip(init_paper           [search_paper_index],search_paper_index)}
    return np.array(sorted(cos.items(), key=lambda x: x[0], reverse=True))[:,1]

In [14]:
search("北京爱情故事，女主角病死")

北京 [78]
爱情 [60, 92, 62, 84]
[78 60 92 62 84]


array([' 故事发生在上世纪七十年代，寄人篱下的少女何小萍（苗苗饰）经过重重试炼终于加入了梦寐以求的文工团，哪知道这里和她曾经待过的那些地方并无不同，她依然得忍受遭人唾弃和欺侮的生活。唯一给过她温暖的，是刘峰（黄轩饰），不仅仅是对何小萍，被称为“活雷锋”的刘峰把自己的爱情和温暖无私的奉献给每一个人。 \n',
       ' 编号为223的警察（金城武）失恋后患上失恋综合症，在与金发女杀手（林青霞）擦肩而过又离奇相遇并有了一晚温情后，原本以为包括“爱情”在内的所有东西都有保质期的他意外地迎来心灵的短暂温暖。可是，他们的爱情还是结束了。 \n',
       ' 许多年前，亚特兰蒂斯女王（妮可·基德曼NicoleKidman饰）和人类相知相恋，共同孕育了爱情的结晶——后来被陆地人称为海王的亚瑟·库瑞（杰森·莫玛JasonMomoa饰）。在成长的过程中，亚瑟接受海底导师维科（威廉·达福WillemDafoe饰）的严苛训练，时刻渴望去看望母亲，然而作为混血的私生子这却是奢望。与此同时，亚瑟的同母异父兄弟奥姆（帕特里克·威尔森PatrickWilson饰）成为亚特兰蒂斯的国王，他不满陆地人类对大海的荼毒与污染，遂谋划联合其他海底王国发动对陆地的全面战争。为了阻止他的野心，维科和奥姆的未婚妻湄拉（艾梅柏·希尔德AmberHeard饰）将亚瑟带到海底世界。 \n',
       ' 结婚五周年纪念日的早上，尼克·邓恩（本·阿弗莱克BenAffleck饰）来到妹妹玛戈（凯莉·库恩CarrieCoon饰）的酒吧，咒骂抱怨那个曾经彼此恩爱缠绵的妻子艾米（罗莎蒙德·派克RosamundPike饰）以及全然看不见希望的婚姻。当他返回家中时，却发现客厅留下了暴行的痕迹，而妻子竟不见了踪影。女探员朗达·邦妮（金·迪肯斯KimDickens饰）接到报案后赶来调查，而现场留下的种种蛛丝马迹似乎昭示着这并非是一件寻常的失踪案，其背后或许隐藏着裂变于夫妻之情的谋杀罪行。艾米的失踪通过媒体大肆渲染和妄加揣测很快闻名全国，品行不端的尼克被推上风口浪尖，至今不见踪影的爱人对他进行无情审判，你侬我侬的甜言蜜语早已化作以血洗血的复仇与折磨…… \n',
       ' 老炮儿在北京话中，专指提笼遛鸟，无所事事的老混混儿。 \n'], dtype='<U323')